In [5]:
#from MuZeroParallel import MuZero as MuZeroP

from MuZero import MuZero

from GridWorldOption import GridWorldOption


from GridWorld import GridWorld

from mcts.MCTS import MCTS
import numpy as np
import matplotlib.pyplot as plt

from random import choice
from copy import deepcopy
import time

import pickle
import pandas as pd

from matplotlib.pyplot import figure
import itertools
from tqdm import tqdm


MAP_NAME = './maps/door.map'

options = [
      GridWorldOption((0, 0),  {'all'}, 0, 0),
      GridWorldOption((0, 0),  {'all'}, 1, 1),
      GridWorldOption((0, 0),  {'all'}, 2, 2),
      GridWorldOption((0, 0),  {'all'}, 3, 3)
]

with open(MAP_NAME) as f:
    the_map = f.read()
    
first_room_pos = [(i,j) for i in range(6) for j in range(8)]
second_room_pos = [(i,j) for i in range(6) for j in range(9, 15)]
third_room_pos = [(i,j) for i in range(7,13) for j in range(8)]
fourth_room_pos = [(i,j) for i in range(7,13) for j in range(9,15)]

full_options = [
  # primitives, (0,0) is meaningless
  GridWorldOption((0, 0),   {'all'}, 0, 0),
  GridWorldOption((0, 0),   {'all'}, 1, 1),
  GridWorldOption((0, 0),   {'all'}, 2, 2),
  GridWorldOption((0, 0),   {'all'}, 3, 3),
  GridWorldOption((3,8),  set(first_room_pos + second_room_pos + [(6,3)] + [(6,13)]), 4),
  GridWorldOption((6,3),  set(first_room_pos + third_room_pos + [(3,8)] + [(11,8)]), 5),
  GridWorldOption((6,13), set(second_room_pos + fourth_room_pos + [(3,8)] + [(11,8)]), 6),
  GridWorldOption((11,8), set(third_room_pos + fourth_room_pos + [(6,3)] + [(6,13)]), 7),
]    

env = GridWorld(the_map)

G_model = {}
for position in tqdm(env.possible_positions):
    for option in full_options:
        if option.is_valid(position):
            
            test_env = GridWorld(the_map)
            test_env.cur_x = position[0]
            test_env.cur_y = position[1]
            rewards = []
            while True:
                action, should_break = option.get_action(test_env)
                
                if action == -1:
                    break
                
                next_position, r, _ = test_env.step(action)
                rewards.append(r)
                
                if should_break:
                    break
                
            G_model[(position, option)] = (next_position, rewards)


100%|███████████████████████████████████████████████████████████████████████████████████████████| 172/172 [00:00<00:00, 1990.22it/s]


In [6]:
first_room_pos = [(i,j) for i in range(6) for j in range(8)]
second_room_pos = [(i,j) for i in range(6) for j in range(9, 15)]
third_room_pos = [(i,j) for i in range(7,13) for j in range(8)]
fourth_room_pos = [(i,j) for i in range(7,13) for j in range(9,15)]

options = [
  # primitives, (0,0) is meaningless
  GridWorldOption((0, 0),   {'all'}, 0, 0),
  GridWorldOption((0, 0),   {'all'}, 1, 1),
  GridWorldOption((0, 0),   {'all'}, 2, 2),
  GridWorldOption((0, 0),   {'all'}, 3, 3),
  GridWorldOption((3,8),  set(first_room_pos + second_room_pos + [(6,3)] + [(6,13)]), 4),
  GridWorldOption((6,3),  set(first_room_pos + third_room_pos + [(3,8)] + [(11,8)]), 5),
  GridWorldOption((6,13), set(second_room_pos + fourth_room_pos + [(3,8)] + [(11,8)]), 6),
  GridWorldOption((11,8), set(third_room_pos + fourth_room_pos + [(6,3)] + [(6,13)]), 7),
]

with open(MAP_NAME) as f:
    the_map = f.read()
    
env = GridWorld(the_map)
mu_opt = MuZero(env, options)

mu_opt.g.model_table = G_model


In [7]:
s0 = env.reset()

mcts = MCTS(s0, mu_opt.f, mu_opt.g, mu_opt.options)
mcts.run_sim(1000)

array([0.167, 0.167, 0.166, 0.167, 0.166, 0.167, 0.   , 0.   ])

In [8]:
mcts.info()

Expanded:  True
Leaf:  False
hidden state (1, 1)
Option 0
N(s, o) 167
Q(s, o) -3.643551353293415
P(s, o) 0.16666666666666666
Q'(s, o):  0.0
UCT exploration:  0.18823081310526066
Prior regulation:  1.299681472999111
U(s, o) 0.04077335007341092
Option 1
N(s, o) 167
Q(s, o) -3.643551353293415
P(s, o) 0.16666666666666666
Q'(s, o):  0.0
UCT exploration:  0.18823081310526066
Prior regulation:  1.299681472999111
U(s, o) 0.04077335007341092
Option 2
N(s, o) 166
Q(s, o) -3.6417631144578335
P(s, o) 0.16666666666666666
Q'(s, o):  0.000676453223938167
UCT exploration:  0.18935794372265743
Prior regulation:  1.299681472999111
U(s, o) 0.041693955094195864
Option 3
N(s, o) 167
Q(s, o) -3.643551353293415
P(s, o) 0.16666666666666666
Q'(s, o):  0.0
UCT exploration:  0.18823081310526066
Prior regulation:  1.299681472999111
U(s, o) 0.04077335007341092
Option 4
N(s, o) 166
Q(s, o) -3.6417631144578335
P(s, o) 0.16666666666666666
Q'(s, o):  0.000676453223938167
UCT exploration:  0.18935794372265743
Prior reg